In [1]:
%load_ext memory_profiler

In [6]:
import sys
sys.path.append("../")

In [79]:
%%writefile memoryprofiling_LBFGS.py

from NuMPI.Optimization import LBFGS
from tests.MPI_minimization_problems import MPI_Quadratic
import numpy as np
import scipy
from NuMPI import MPI
from NuMPI.Tools import Reduction
from memory_profiler import profile
import sys
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nprocs= comm.Get_size()
pnp = Reduction(comm)
print(sys.argv)
n= int(sys.argv[1])
print(n)

fp=open("LBFGS_memory_profile_n{}_nprocs{}_rank{}.log".format(n, nprocs, rank), "w")
LBFGS = profile(stream=fp)(LBFGS)

Objective = MPI_Quadratic(n, pnp=pnp)
result = LBFGS(Objective.f,Objective.startpoint(),
                jac=Objective.grad,
                options ={"gtol":1e-6,"pnp":Objective.pnp})
fp.close()
assert result.success
print(result.nit)

Overwriting memoryprofiling_LBFGS.py


In [114]:
!pip install parse

  Stored in directory: /Users/antoines/Library/Caches/pip/wheels/9f/62/d1/c46b7452aa0b2c838080bdd462110cd6c61890151f916aa743
Successfully built parse
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [115]:
from parse import parse
from glob import glob

In [117]:

for fn in glob("LBFGS_memory_profile_n*_nprocs*_rank*.log"):
    n, nprocs, rank = parse("LBFGS_memory_profile_n{:d}_nprocs{:d}_rank{:d}.log", fn)
    
    

In [164]:
ls *.log

LBFGS_memory_profile_n10000000_nprocs2_rank0.log
LBFGS_memory_profile_n10000000_nprocs2_rank1.log
LBFGS_memory_profile_n1000000_nprocs2_rank0.log
LBFGS_memory_profile_n1000000_nprocs2_rank1.log
LBFGS_memory_profile_n100000_nprocs2_rank0.log
LBFGS_memory_profile_n100000_nprocs2_rank1.log
LBFGS_memory_profile_n10000_nprocs2_rank0.log
LBFGS_memory_profile_n10000_nprocs2_rank1.log


In [122]:
import pandas as pd

In [133]:
fn

'LBFGS_memory_profile_n10000_nprocs2_rank0.log'

In [145]:

text = !sed -n -e  "/^   [0-9]* [ ]*[0-9]*[.][0-9]* MiB/p" {fn}

In [165]:
fn="LBFGS_memory_profile_n10000000_nprocs2_rank0.log"
import re 
expr = re.compile(r"^\s*([0-9]*) [0-9]* [ ]*([0-9]*[.][0-9]*) MiB[ ]*([0-9]*[.][0-9]*) MiB [ ]*(.*)$" )
for line in text:
    print(line)
    ln, memusage, memincr, code = expr.match(line).groups()
    row = pd.DataFrame.from_records(dict(
    linenumber=[int(ln)], 
    mem_usage=float(memusage), 
    mem_incr=float(memincr), 
    code=code
    ),)

   109     55.8 MiB      0.0 MiB       if callback is None:
   110     55.8 MiB      0.0 MiB           callback=donothing
   113     55.8 MiB      0.0 MiB       if jac is True:
   128     55.8 MiB      0.0 MiB       elif jac is False:
   133     58.1 MiB      0.1 MiB           def fun_grad(x):
   146     58.1 MiB      0.3 MiB               f = fun(x)
   147     58.1 MiB      0.0 MiB               grad = jac(x).reshape((-1,1))
   148     58.1 MiB      0.0 MiB               return f, grad
   152     55.8 MiB      0.0 MiB       original_shape = x.shape
   154     55.8 MiB      0.0 MiB       x = x.reshape((-1,1))
   156     55.8 MiB      0.0 MiB       if x_old is None:
   157     55.8 MiB      0.0 MiB           x_old = x.copy()
   159     55.8 MiB      0.0 MiB               steepest_descent_wolfe2(x_old, fun_grad, pnp=pnp, maxiter=maxls,
   160     56.3 MiB      0.0 MiB                                       **linesearch_options)
   166     56.3 MiB      0.0 MiB       iterates = list()
   1

('348', '58.1', '0.0', 'YTgrad_prev = YTgrad.copy()')

In [132]:
!cat {fn} | sed "/^[0-9]/p "

Filename: /Users/antoines/NuMPI/NuMPI/Optimization/MPI_LBFGS_Matrix_H.py

Line #    Mem usage    Increment   Line Contents
    68     55.8 MiB     55.8 MiB   def LBFGS(fun, x, args=(), jac=None, x_old=None, maxcor=10, gtol = 1e-5, ftol=2.2e-9, maxiter=15000,
    69                                       maxls=20, linesearch_options=dict(c1=1e-3, c2=0.9), pnp=Reduction(MPI.COMM_WORLD),
    70                                       store_iterates=None, printdb=donothing, callback=None, **options):
    71                                 """
    72                             
    73                                 convergence if |grad|_{\infty} <= gtol or <= ftol is satisfied
    74                             
    75                                 Parameters
    76                                 ----------
    77                                 fun
    78                                 x
    79                                 args
    80                                 jac
    81       

In [125]:
pd.DataFrame.from_csv(fn, sep=" ")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


ParserError: Error tokenizing data. C error: Expected 15 fields in line 5, saw 31


In [97]:
%cd helpers

/Users/antoines/NuMPI/helpers


In [1]:
import subprocess
output = subprocess.check_output("D=$PWD;"
                                 "cd ../../;"
                                 "source env.sh;"
                                 "cd $D;"
                                 "mpirun -np 2 python memoryprofiling_LBFGS.py 10000",shell=True, timeout=120,stderr=subprocess.STDOUT ).decode()

CalledProcessError: Command 'D=$PWD;cd ../../;source env.sh;cd $D;mpirun -np 2 python memoryprofiling_LBFGS.py 10000' returned non-zero exit status 2.

In [93]:
output

NameError: name 'output' is not defined

In [88]:
ls

LBFGS_Matrix_H_Verbose.py
LBFGS_Matrix_Hk.ipynb
LBFGS_Matrix_Hk_Opt1.ipynb
LBFGS_iterative.ipynb
LBFGS_memory_profile_n10000_nprocs2_rank0.log
LBFGS_memory_profile_n10000_nprocs2_rank1.log
Memory profiling LBFGS line by line.ipynb
Numpy Matrix Operations.ipynb
__pycache__/
copyright.py
learn_scipy_linesearch.py
line_profiing_LBFGSB.ipynb
memoryprofiling_LBFGS.py
nb_fevals_scipy_vs_numpy.py
plot_helpers.py
profiling.py
replace_header.py
solve_Rosenbrock.py
temp_LBFGS_performance_test.py
timing basic numpy operations.ipynb
update_license_headers.sh


In [65]:
!sublime LBFGS_memory_profile_n1000_nprocs2_rank0.log